# Predicting Optimal Fertilizers

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

/kaggle/input/playground-series-s5e6/sample_submission.csv
/kaggle/input/playground-series-s5e6/train.csv
/kaggle/input/playground-series-s5e6/test.csv


Private-fertilizer-predic-playground-series-s5-e6 Version 1 notebook Full EDA

Public notebook: https://www.kaggle.com/code/les1781/optimal-fertilizer-predict-playground-series-s5-e6

# Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import (
    LabelEncoder,    
    OneHotEncoder,
    StandardScaler
)
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    f1_score, 
    accuracy_score,
    make_scorer
)
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance

# Initial analysis

In [3]:
# We load the data

fertilizers_train = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv", index_col="id")

In [4]:
fertilizers_train.describe().T

,count,mean,std,min,25%,50%,75%,max
Temparature,750000.0,31.503565,4.025574,25.0,28.0,32.0,35.0,38.0
Humidity,750000.0,61.038912,6.647695,50.0,55.0,61.0,67.0,72.0
Moisture,750000.0,45.184147,11.794594,25.0,35.0,45.0,55.0,65.0
Nitrogen,750000.0,23.093808,11.216125,4.0,13.0,23.0,33.0,42.0
Potassium,750000.0,9.478296,5.765622,0.0,4.0,9.0,14.0,19.0
Phosphorous,750000.0,21.073227,12.346831,0.0,10.0,21.0,32.0,42.0


In [5]:
fertilizers_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Temparature      750000 non-null  int64 
 1   Humidity         750000 non-null  int64 
 2   Moisture         750000 non-null  int64 
 3   Soil Type        750000 non-null  object
 4   Crop Type        750000 non-null  object
 5   Nitrogen         750000 non-null  int64 
 6   Potassium        750000 non-null  int64 
 7   Phosphorous      750000 non-null  int64 
 8   Fertilizer Name  750000 non-null  object
dtypes: int64(6), object(3)
memory usage: 57.2+ MB


# Data wrangling

In [6]:
# We make a copy of the original dataset

fertilizers_new = fertilizers_train.copy()

In [7]:
# We confirm that there is no null values

null_values = pd.DataFrame(
        {f"Null Data" : fertilizers_new.isnull().sum(), 
         "Percentage" : (fertilizers_new.isnull().sum()) / (len(fertilizers_new)) * (100)})

null_values

,Null Data,Percentage
Temparature,0,0.0
Humidity,0,0.0
Moisture,0,0.0
Soil Type,0,0.0
Crop Type,0,0.0
Nitrogen,0,0.0
Potassium,0,0.0
Phosphorous,0,0.0
Fertilizer Name,0,0.0


In [8]:
# We check the duplicate data found

print(f"Length: {len(fertilizers_new.duplicated())}")
print(f"Duplicates: {fertilizers_new.duplicated().sum()}")

Length: 750000
Duplicates: 0


In [9]:
'''
eval_out = sns.PairGrid(fertilizers_new, palette=sns.light_palette("seagreen"))
eval_out.map(sns.boxplot)
eval_out.tick_params(axis="both", labelbottom=False)
'''

'\neval_out = sns.PairGrid(fertilizers_new, palette=sns.light_palette("seagreen"))\neval_out.map(sns.boxplot)\neval_out.tick_params(axis="both", labelbottom=False)\n'

# Data Preprocessing

In [10]:
fertilizers_end = fertilizers_new.copy()

## Feature Engineering

First approach with original variables only.

## Encoding

In [11]:
# We apply LabelEncoder to the target variable

le = LabelEncoder()

fertilizers_end["Fertilizer Name"] = le.fit_transform(fertilizers_end["Fertilizer Name"])

In [12]:
le_values = le.classes_

In [13]:
# We create a df with the encoded categorical variables

cat_cols = fertilizers_end[["Soil Type", "Crop Type"]]
rest_cols = fertilizers_end.drop(columns=["Soil Type", "Crop Type"])
encoder = OneHotEncoder(sparse=False, drop="if_binary").set_output(transform="pandas")
cat_enc = encoder.fit_transform(cat_cols)
df_encoded = pd.concat([rest_cols, cat_enc], axis=1)

In [14]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 23 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Temparature            750000 non-null  int64  
 1   Humidity               750000 non-null  int64  
 2   Moisture               750000 non-null  int64  
 3   Nitrogen               750000 non-null  int64  
 4   Potassium              750000 non-null  int64  
 5   Phosphorous            750000 non-null  int64  
 6   Fertilizer Name        750000 non-null  int64  
 7   Soil Type_Black        750000 non-null  float64
 8   Soil Type_Clayey       750000 non-null  float64
 9   Soil Type_Loamy        750000 non-null  float64
 10  Soil Type_Red          750000 non-null  float64
 11  Soil Type_Sandy        750000 non-null  float64
 12  Crop Type_Barley       750000 non-null  float64
 13  Crop Type_Cotton       750000 non-null  float64
 14  Crop Type_Ground Nuts  750000 non-null  f

In [15]:
#df_encoded.corr(numeric_only=True).round(2)

## Scaling

In [16]:
df_encoded.describe().T

,count,mean,std,min,25%,50%,75%,max
Temparature,750000.0,31.503565,4.025574,25.0,28.0,32.0,35.0,38.0
Humidity,750000.0,61.038912,6.647695,50.0,55.0,61.0,67.0,72.0
Moisture,750000.0,45.184147,11.794594,25.0,35.0,45.0,55.0,65.0
Nitrogen,750000.0,23.093808,11.216125,4.0,13.0,23.0,33.0,42.0
Potassium,750000.0,9.478296,5.765622,0.0,4.0,9.0,14.0,19.0
Phosphorous,750000.0,21.073227,12.346831,0.0,10.0,21.0,32.0,42.0
Fertilizer Name,750000.0,2.859791,1.967025,0.0,1.0,3.0,4.0,6.0
Soil Type_Black,750000.0,0.201275,0.400953,0.0,0.0,0.0,0.0,1.0
Soil Type_Clayey,750000.0,0.197843,0.398373,0.0,0.0,0.0,0.0,1.0
Soil Type_Loamy,750000.0,0.194467,0.395790,0.0,0.0,0.0,0.0,1.0


In [17]:
# We separate the target variable from the features

x_fertilizers = df_encoded.drop(columns="Fertilizer Name")
y_fertilizers = df_encoded["Fertilizer Name"]

In [18]:
# Numerical variables to scale

fertilizer_numeric = x_fertilizers[[
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous"
]]

scaler = StandardScaler().set_output(transform="pandas")
scale_num = scaler.fit_transform(fertilizer_numeric)

# We create a df with the remaining variables

fertilizer_rest = x_fertilizers.drop(columns=[
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous"
])

# We concatenate the dataframes

x_end = pd.concat([scale_num, fertilizer_rest], axis=1)

In [19]:
x_end.info()

<class 'pandas.core.frame.DataFrame'>
Index: 750000 entries, 0 to 749999
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Temparature            750000 non-null  float64
 1   Humidity               750000 non-null  float64
 2   Moisture               750000 non-null  float64
 3   Nitrogen               750000 non-null  float64
 4   Potassium              750000 non-null  float64
 5   Phosphorous            750000 non-null  float64
 6   Soil Type_Black        750000 non-null  float64
 7   Soil Type_Clayey       750000 non-null  float64
 8   Soil Type_Loamy        750000 non-null  float64
 9   Soil Type_Red          750000 non-null  float64
 10  Soil Type_Sandy        750000 non-null  float64
 11  Crop Type_Barley       750000 non-null  float64
 12  Crop Type_Cotton       750000 non-null  float64
 13  Crop Type_Ground Nuts  750000 non-null  float64
 14  Crop Type_Maize        750000 non-null  f

In [20]:
x_end.describe().T

,count,mean,std,min,25%,50%,75%,max
Temparature,750000.0,-2.430625e-16,1.000001,-1.615563,-0.870328,0.123320,0.868556,1.613792
Humidity,750000.0,-5.286627e-16,1.000001,-1.660563,-0.908422,-0.005853,0.896716,1.648856
Moisture,750000.0,2.707263e-16,1.000001,-1.711306,-0.863459,-0.015613,0.832234,1.680080
Nitrogen,750000.0,3.865352e-17,1.000001,-1.702354,-0.899938,-0.008364,0.883210,1.685627
Potassium,750000.0,-5.222015e-17,1.000001,-1.643934,-0.950166,-0.082957,0.784253,1.651463
Phosphorous,750000.0,-2.046363e-18,1.000001,-1.706773,-0.896848,-0.005931,0.884987,1.694912
Soil Type_Black,750000.0,2.012747e-01,0.400953,0.000000,0.000000,0.000000,0.000000,1.000000
Soil Type_Clayey,750000.0,1.978427e-01,0.398373,0.000000,0.000000,0.000000,0.000000,1.000000
Soil Type_Loamy,750000.0,1.944667e-01,0.395790,0.000000,0.000000,0.000000,0.000000,1.000000
Soil Type_Red,750000.0,1.974693e-01,0.398090,0.000000,0.000000,0.000000,0.000000,1.000000


## Feature Selection

In [21]:
'''
fertilizers_scores = mutual_info_classif(x_end, y_fertilizers)
fertilizers_scores = pd.Series(fertilizers_scores, name="Fertilizers MI Scores", index=x_end.columns)
fertilizers_scores = fertilizers_scores.sort_values(ascending=False)
fertilizers_scores
'''

'\nfertilizers_scores = mutual_info_classif(x_end, y_fertilizers)\nfertilizers_scores = pd.Series(fertilizers_scores, name="Fertilizers MI Scores", index=x_end.columns)\nfertilizers_scores = fertilizers_scores.sort_values(ascending=False)\nfertilizers_scores\n'

First feature analysis:

    Moisture                 0.004975
    Soil Type_Black          0.004838
    Phosphorous              0.004139
    Soil Type_Clayey         0.003320
    Soil Type_Loamy          0.003201
    Nitrogen                 0.002672
    Soil Type_Sandy          0.002432
    Soil Type_Red            0.002427
    Crop Type_Oil seeds      0.002218
    Potassium                0.001786
    Temparature              0.001252
    Crop Type_Paddy          0.001237
    Crop Type_Tobacco        0.001203
    Crop Type_Pulses         0.001140
    Crop Type_Maize          0.001094
    Crop Type_Wheat          0.000915
    Crop Type_Barley         0.000832
    Crop Type_Cotton         0.000788
    Humidity                 0.000540
    Crop Type_Ground Nuts    0.000466
    Crop Type_Sugarcane      0.000300
    Crop Type_Millets        0.000000

In [22]:
'''
scores = fertilizers_scores.sort_values(ascending=True)
width = np.arange(len(fertilizers_scores))
ticks = list(fertilizers_scores.index)
plt.barh(width, fertilizers_scores)
plt.yticks(width, ticks)
plt.title("Mutual Information Scores")
plt.figure(dpi=100, figsize=(8, 5))
plt.show()
'''

'\nscores = fertilizers_scores.sort_values(ascending=True)\nwidth = np.arange(len(fertilizers_scores))\nticks = list(fertilizers_scores.index)\nplt.barh(width, fertilizers_scores)\nplt.yticks(width, ticks)\nplt.title("Mutual Information Scores")\nplt.figure(dpi=100, figsize=(8, 5))\nplt.show()\n'

In [23]:
#x_end = x_end.drop(columns=[])

# Model Selection

In [24]:
# We separate the data into training and validation sets

x_train, x_val, y_train, y_val = (
    train_test_split(x_end, y_fertilizers, test_size=0.2, random_state=42)
)

In [25]:
# We review the balance of the target variable

values_counts = np.asarray(np.unique(y_train, return_counts=True))

print(values_counts)

[[    0     1     2     3     4     5     6]
 [91046 91797 89689 88879 88774 75712 74103]]


In [26]:
# Common function to calculate AP@K and then MAP@K

def apk(actual, predicted, k):

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score

def mapk(actual, predicted, k):

    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [27]:
# Function for an initial evaluation of the model

def evaluator(model, val_x, val_y, enc, k, model_name):
    
    y_pred_prob = model.predict_proba(val_x)
    top_3_indices_val = np.argsort(-y_pred_prob, axis=1)[:, :3]
    
    predicted_val = []
    
    for row_indices in top_3_indices_val:
        names = enc.inverse_transform(row_indices)
        predicted_val.append(list(names))
    
    actual_val = [[enc.inverse_transform([label])[0]] for label in val_y]
    map_3_score_val = mapk(actual_val, predicted_val, k=k)
    
    print(f"\n{model_name} MAP@3 Score: {map_3_score_val:.4f}")

## LogisticRegression

In [28]:
# We create the model instance

lrc = LogisticRegression(multi_class="multinomial")

# Train the model with the data

lrc.fit(x_train, y_train)

LogisticRegression(multi_class='multinomial')

In [29]:
evaluator(lrc, x_val, y_val, le, 3, "LogisticRegression")


LogisticRegression MAP@3 Score: 0.2870


# Modeling

In [30]:
final_model = lrc

final_model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'multinomial',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [31]:
# Permutation Importance

perm_importance = permutation_importance(final_model, x_val, y_val, n_repeats=30, random_state=42, n_jobs=-1)
perm_importance_df = pd.DataFrame({
    'Feature': x_end.columns,
    'Importance Mean': perm_importance.importances_mean,
    'Importance Std': perm_importance.importances_std
})
print("\nPermutation Importance:")
print(perm_importance_df.sort_values(by='Importance Mean', ascending=False))


Permutation Importance:
                  Feature  Importance Mean  Importance Std
2                Moisture         0.001306        0.000507
17        Crop Type_Paddy         0.000888        0.000325
18       Crop Type_Pulses         0.000798        0.000282
5             Phosphorous         0.000743        0.000328
20      Crop Type_Tobacco         0.000674        0.000247
10        Soil Type_Sandy         0.000447        0.000374
21        Crop Type_Wheat         0.000376        0.000367
6         Soil Type_Black         0.000363        0.000349
1                Humidity         0.000343        0.000448
11       Crop Type_Barley         0.000336        0.000276
4               Potassium         0.000294        0.000390
9           Soil Type_Red         0.000222        0.000306
16    Crop Type_Oil seeds         0.000214        0.000257
13  Crop Type_Ground Nuts         0.000162        0.000211
15      Crop Type_Millets         0.000102        0.000296
7        Soil Type_Clayey      

# Test data

In [32]:
# We load the test data and submission sample data

df_test = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")

fertilizer_sample = pd.read_csv("/kaggle/input/playground-series-s5e6/sample_submission.csv")

In [33]:
# We check the shape

print(f"Shape: {df_test.shape}")

Shape: (250000, 9)


In [34]:
df_test.head()

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,750000,31,70,52,Sandy,Wheat,34,11,24
1,750001,27,62,45,Red,Sugarcane,30,14,15
2,750002,28,72,28,Clayey,Ground Nuts,14,15,4
3,750003,37,53,57,Black,Ground Nuts,18,17,36
4,750004,31,55,32,Red,Pulses,13,19,14


In [35]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           250000 non-null  int64 
 1   Temparature  250000 non-null  int64 
 2   Humidity     250000 non-null  int64 
 3   Moisture     250000 non-null  int64 
 4   Soil Type    250000 non-null  object
 5   Crop Type    250000 non-null  object
 6   Nitrogen     250000 non-null  int64 
 7   Potassium    250000 non-null  int64 
 8   Phosphorous  250000 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 17.2+ MB


In [36]:
df_test.describe().T

,count,mean,std,min,25%,50%,75%,max
id,250000.0,874999.500000,72168.927986,750000.0,812499.75,874999.5,937499.25,999999.0
Temparature,250000.0,31.491648,4.024093,25.0,28.00,31.0,35.00,38.0
Humidity,250000.0,61.045120,6.636372,50.0,55.00,61.0,67.00,72.0
Moisture,250000.0,45.190444,11.793167,25.0,35.00,45.0,55.00,65.0
Nitrogen,250000.0,23.139612,11.215956,4.0,13.00,23.0,33.00,42.0
Potassium,250000.0,9.487764,5.766860,0.0,4.00,10.0,14.00,19.0
Phosphorous,250000.0,21.122060,12.380870,0.0,10.00,21.0,32.00,42.0


In [37]:
df_test.describe(exclude = np.number)

,Soil Type,Crop Type
count,250000,250000
unique,5,11
top,Sandy,Paddy
freq,52323,28332


## Data wrangling & Preprocessing

In [38]:
# We check that no duplicate data is found

print(f"Length: {len(df_test.duplicated())}")

print(f"Duplicates: {df_test.duplicated().sum()}")

Length: 250000
Duplicates: 0


In [39]:
# We confirm that there is no null values

null_values_test = pd.DataFrame(
        {f"Null Data" : df_test.isnull().sum(), 
         "Percentage" : (df_test.isnull().sum()) / (len(df_test)) * (100)})

null_values_test

,Null Data,Percentage
id,0,0.0
Temparature,0,0.0
Humidity,0,0.0
Moisture,0,0.0
Soil Type,0,0.0
Crop Type,0,0.0
Nitrogen,0,0.0
Potassium,0,0.0
Phosphorous,0,0.0


In [40]:
# We start by removing the variables that we will not use

test_new = df_test.drop(columns=["id"])

In [41]:
# We encode categorical variables

test_cat_cols = test_new[["Soil Type", "Crop Type"]]
test_rest_cols = test_new.drop(columns=["Soil Type", "Crop Type"])
test_cat_enc = encoder.fit_transform(test_cat_cols)
test_encoded = pd.concat([test_rest_cols, test_cat_enc], axis=1)

In [42]:
test_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Temparature            250000 non-null  int64  
 1   Humidity               250000 non-null  int64  
 2   Moisture               250000 non-null  int64  
 3   Nitrogen               250000 non-null  int64  
 4   Potassium              250000 non-null  int64  
 5   Phosphorous            250000 non-null  int64  
 6   Soil Type_Black        250000 non-null  float64
 7   Soil Type_Clayey       250000 non-null  float64
 8   Soil Type_Loamy        250000 non-null  float64
 9   Soil Type_Red          250000 non-null  float64
 10  Soil Type_Sandy        250000 non-null  float64
 11  Crop Type_Barley       250000 non-null  float64
 12  Crop Type_Cotton       250000 non-null  float64
 13  Crop Type_Ground Nuts  250000 non-null  float64
 14  Crop Type_Maize        250000 non-nu

In [43]:
# Numerical variables to scale

test_numeric = test_encoded[[
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous"
]]
test_scale_num = scaler.transform(test_numeric)
test_rest = test_encoded.drop(columns=[
    "Temparature",
    "Humidity",
    "Moisture",
    "Nitrogen",
    "Potassium",
    "Phosphorous"
])
test_end = pd.concat([test_scale_num, test_rest], axis=1)

In [44]:
test_end.describe().T

,count,mean,std,min,25%,50%,75%,max
Temparature,250000.0,-0.002960,0.999633,-1.615563,-0.870328,-0.125092,0.868556,1.613792
Humidity,250000.0,0.000934,0.998297,-1.660563,-0.908422,-0.005853,0.896716,1.648856
Moisture,250000.0,0.000534,0.999880,-1.711306,-0.863459,-0.015613,0.832234,1.680080
Nitrogen,250000.0,0.004084,0.999986,-1.702354,-0.899938,-0.008364,0.883210,1.685627
Potassium,250000.0,0.001642,1.000215,-1.643934,-0.950166,0.090485,0.784253,1.651463
Phosphorous,250000.0,0.003955,1.002758,-1.706773,-0.896848,-0.005931,0.884987,1.694912
Soil Type_Black,250000.0,0.200588,0.400441,0.000000,0.000000,0.000000,0.000000,1.000000
Soil Type_Clayey,250000.0,0.199168,0.399375,0.000000,0.000000,0.000000,0.000000,1.000000
Soil Type_Loamy,250000.0,0.193516,0.395055,0.000000,0.000000,0.000000,0.000000,1.000000
Soil Type_Red,250000.0,0.197436,0.398065,0.000000,0.000000,0.000000,0.000000,1.000000


In [45]:
# We remove the variables that we will not use

#test_end_new = test_end.drop(columns=[])

In [46]:
test_end.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Temparature            250000 non-null  float64
 1   Humidity               250000 non-null  float64
 2   Moisture               250000 non-null  float64
 3   Nitrogen               250000 non-null  float64
 4   Potassium              250000 non-null  float64
 5   Phosphorous            250000 non-null  float64
 6   Soil Type_Black        250000 non-null  float64
 7   Soil Type_Clayey       250000 non-null  float64
 8   Soil Type_Loamy        250000 non-null  float64
 9   Soil Type_Red          250000 non-null  float64
 10  Soil Type_Sandy        250000 non-null  float64
 11  Crop Type_Barley       250000 non-null  float64
 12  Crop Type_Cotton       250000 non-null  float64
 13  Crop Type_Ground Nuts  250000 non-null  float64
 14  Crop Type_Maize        250000 non-nu

## Apply the Model & Submission File

In [47]:
# We apply the trained model

test_pred_prob = final_model.predict_proba(test_end)
test_top_3 = np.argsort(-test_pred_prob, axis=1)[:, :3]

top_3_fertilizer_names = []
for row_indices in test_top_3:
    valid_indices = [idx for idx in row_indices if idx < len(le.classes_)]
    names = le.inverse_transform(valid_indices)
    top_3_fertilizer_names.append(names)

formatted_predictions = [" ".join(names) for names in top_3_fertilizer_names]

In [48]:
# We review the result

print("Total predictions: ", len(formatted_predictions), "\n")

Total predictions:  250000 



In [49]:
# We create the dataframe

fertilizer_submission = pd.DataFrame({
    "id" : df_test["id"], 
    "Fertilizer Name" : formatted_predictions
})

fertilizer_submission.head()

,id,Fertilizer Name
0,750000,20-20 10-26-26 DAP
1,750001,14-35-14 20-20 10-26-26
2,750002,17-17-17 20-20 28-28
3,750003,14-35-14 28-28 20-20
4,750004,17-17-17 14-35-14 20-20


In [50]:
# We compare the results with the sample

print(
    f"Shape Sample Submission: {fertilizer_sample.shape}",
    f"\nShape Fertilizer Submission: {fertilizer_submission.shape}"
)
print("\n", fertilizer_sample.head())

Shape Sample Submission: (250000, 2) 
Shape Fertilizer Submission: (250000, 2)

        id         Fertilizer Name
0  750000  14-35-14 10-26-26 Urea
1  750001  14-35-14 10-26-26 Urea
2  750002  14-35-14 10-26-26 Urea
3  750003  14-35-14 10-26-26 Urea
4  750004  14-35-14 10-26-26 Urea


In [51]:
# We convert the dataframe to a csv file

fertilizer_submission.to_csv("submission.csv", index=False)